In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# 저장된 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('./koelectra_v3_ner_model5')

In [2]:
def make_token_input_data(sentence, tokenizer, max_length=64):
    
    """KoElectra모델에 넣을 데이터 준비"""

    tokens = tokenizer.tokenize(sentence)

    list_text = []
    text_token = []

    for token in tokens:
        if token.startswith("##"):  # 특수 기호 제거
            list_text[-1] += token[2:]
        else:
            list_text.append(list(token))

    for a in list_text:
        text_token.extend(a)

    encodings = tokenizer(
        text_token, 
        is_split_into_words=True, 
        padding=True, 
        truncation=True, 
        max_length=max_length,
        return_tensors='pt'
    )

    print(f"list_text: {list_text}")
    print(f"text_token: {text_token}")
    print(f"encodings: {encodings}")

    return encodings,text_token

In [10]:
# 입력 문장
sentence = '명가 우육면에서 마라 우육면 2개 주문해주고 부추계란 물만두 한개 추가해줘'

# 기본 토큰화 및 인코딩
encoding, text_token = make_token_input_data(sentence, tokenizer)

# input_ids와 attention_mask 그대로 확인
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# print("encoding:", encoding)
print("input_ids:", input_ids)
print("attention_mask:", attention_mask)


list_text: [['명', '가'], ['우', '육', '면', '에', '서'], ['마', '라'], ['우', '육', '면'], ['2', '개'], ['주', '문', '해', '주', '고'], ['부', '추', '계', '란'], ['물', '만', '두'], ['한', '개'], ['추', '가', '해', '줘']]
text_token: ['명', '가', '우', '육', '면', '에', '서', '마', '라', '우', '육', '면', '2', '개', '주', '문', '해', '주', '고', '부', '추', '계', '란', '물', '만', '두', '한', '개', '추', '가', '해', '줘']
encodings: {'input_ids': tensor([[   2, 2675, 2010, 3191, 3222, 2672, 3130, 2914, 2628, 2523, 3191, 3222,
         2672,   22, 2027, 3323, 2693, 3764, 3323, 2075, 2799, 3474, 2073, 2525,
         2695, 2630, 2419, 3757, 2027, 3474, 2010, 3764, 3331,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
input_ids: tensor([[   2, 2675, 2010, 3191, 3222, 2672, 3130, 2914, 2628, 2523, 3191, 3222,
  

In [11]:
# 저장된 모델 로드
model = AutoModelForTokenClassification.from_pretrained('./koelectra_v3_ner_model5')
model.eval()

# 추론
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)  # 가장 높은 확률을 가진 클래스 선택

print(sentence)
print(predictions[0])

명가 우육면에서 마라 우육면 2개 주문해주고 부추계란 물만두 한개 추가해줘
tensor([0, 1, 2, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 5, 6, 0, 0, 0, 0, 0, 3, 0, 0, 0,
        3, 4, 4, 5, 6, 0, 0, 0, 0, 0])


- - -

In [13]:
# 입력 문장
sentence = '명가 우육면에서 마라 우육면 2개 주문해주고 부추계란 물만두 한개 추가해줘'

# 기본 토큰화 및 인코딩
encoding, text_token = make_token_input_data(sentence, tokenizer)

# input_ids와 attention_mask 그대로 확인
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

# print("encoding:", encoding)
print("input_ids:", input_ids)
print("attention_mask:", attention_mask)


list_text: [['명', '가'], ['우', '육', '면', '에', '서'], ['마', '라'], ['우', '육', '면'], ['2', '개'], ['주', '문', '해', '주', '고'], ['부', '추', '계', '란'], ['물', '만', '두'], ['한', '개'], ['추', '가', '해', '줘']]
text_token: ['명', '가', '우', '육', '면', '에', '서', '마', '라', '우', '육', '면', '2', '개', '주', '문', '해', '주', '고', '부', '추', '계', '란', '물', '만', '두', '한', '개', '추', '가', '해', '줘']
encodings: {'input_ids': tensor([[   2, 2675, 2010, 3191, 3222, 2672, 3130, 2914, 2628, 2523, 3191, 3222,
         2672,   22, 2027, 3323, 2693, 3764, 3323, 2075, 2799, 3474, 2073, 2525,
         2695, 2630, 2419, 3757, 2027, 3474, 2010, 3764, 3331,    3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
input_ids: tensor([[   2, 2675, 2010, 3191, 3222, 2672, 3130, 2914, 2628, 2523, 3191, 3222,
  

In [14]:
# 저장된 모델 로드
model = AutoModelForTokenClassification.from_pretrained('./koelectra_v3_ner_model5')
model.eval()

# 추론
with torch.no_grad():
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)  # 가장 높은 확률을 가진 클래스 선택

print(sentence)
print(predictions[0])

명가 우육면에서 마라 우육면 2개 주문해주고 부추계란 물만두 한개 추가해줘
tensor([0, 1, 2, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 5, 6, 0, 0, 0, 0, 0, 3, 0, 0, 0,
        3, 4, 4, 5, 6, 0, 0, 0, 0, 0])


In [16]:
a = predictions[0].tolist()
store = []
food = []
ea = []

text_token

for count,tag in enumerate(a):
    if tag == 1 or tag == 2:
        store.append(text_token[count-1])
    elif tag == 3 or tag == 4:
        if tag == 3 :
            food.append(',')
        food.append(text_token[count-1])
    elif tag == 5 or tag == 6:
        ea.append(text_token[count-1])

print(store)
print(food)
print(ea)

['명', '가', '우', '육', '면']
[',', '마', '라', '우', '육', '면', ',', '부', ',', '물', '만', '두']
['2', '개', '한', '개']


In [17]:
store = [''.join(store)]

temp_food = ''.join(food).split(',')
food = [item.strip() for item in temp_food if item.strip()] 

ea = [''.join(ea[i:i+2]) for i in range(0, len(ea), 2)]

print(store) 
print(temp_food)
print(food) 
print(ea) 

['명가우육면']
['', '마라우육면', '부', '물만두']
['마라우육면', '부', '물만두']
['2개', '한개']
